In [79]:
import os
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import myServices as ms
import models as md
import numpy as np
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,roc_curve, auc, roc_auc_score, f1_score
import joblib

In [ ]:
# to compute ececution time do:
# with timeit():
#     # your code, e.g., 
class timeit(): 
    from datetime import datetime
    def __enter__(self):
        self.tic = self.datetime.now()
    def __exit__(self, *args, **kwargs):
        print('runtime: {}'.format(self.datetime.now() - self.tic))

## Importing and manipulating datasets

In [ ]:
sklearn.metrics.get_scorer_names()

In [81]:
### Cleaning basin1DataSet 
DS = pd.read_csv('datasetBasin1_NoDataFree.csv', index_col = None)
print(DS.info())
# DS = DS.dropna()
# print(DS.info())
# DS.to_csv(, index = None)
# DS.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479538 entries, 0 to 2479537
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   percentage  int64  
 1   disToRiv    float64
 2   TWI         float64
 3   TPI         float64
 4   FAcc        float64
 5   slope       float64
 6   elevation   float64
 7   x_coord     int64  
 8   y_coord     int64  
dtypes: float64(6), int64(3)
memory usage: 170.3 MB
None


In [82]:
print(DS.head())

   percentage   disToRiv       TWI      TPI       FAcc    slope  elevation  \
0           0  583.09521   9.31501  0.03631   62.30688  2.34113   73.51645   
1           0  580.38782  10.49633  0.07143   36.18240  1.66741   73.66300   
2           0  578.46777   6.69994  0.04819   67.29689  3.00253   73.45673   
3           0  576.58478   6.09303 -0.03325  122.67625  3.17653   73.14900   
4           0  577.75427   3.46413 -0.10745   92.17310  1.38605   73.60905   

   x_coord  y_coord  
0   370038  5267978  
1   370033  5267973  
2   370038  5267973  
3   370043  5267973  
4   370028  5267968  


In [87]:
basin1Light = pd.DataFrame(DS[DS['disToRiv']<400])

In [91]:
print(basin1Light.describe())

         percentage      disToRiv           TWI           TPI          FAcc  \
count  2.134268e+06  2.134268e+06  2.134268e+06  2.134268e+06  2.134268e+06   
mean   1.069041e-01  1.556604e+02  9.498964e+00 -4.539183e-04  5.033024e+04   
std    7.114130e-01  1.083439e+02  4.106028e+00  1.970134e-01  1.111771e+06   
min    0.000000e+00  0.000000e+00 -9.252120e+00 -4.857010e+00  2.500000e+01   
25%    0.000000e+00  6.184658e+01  5.566240e+00 -6.273000e-02  8.580688e+01   
50%    0.000000e+00  1.386542e+02  1.025559e+01 -2.600000e-04  2.815533e+02   
75%    0.000000e+00  2.375395e+02  1.286854e+01  6.117000e-02  1.351672e+03   
max    5.000000e+00  3.999062e+02  2.486353e+01  3.891020e+00  6.282450e+07   

              slope     elevation       x_coord       y_coord  
count  2.134268e+06  2.134268e+06  2.134268e+06  2.134268e+06  
mean   4.741202e+00  1.397329e+02  3.650266e+05  5.264290e+06  
std    5.027564e+00  4.454908e+01  3.131664e+03  1.759008e+03  
min    4.700000e-04 -3.685000e-0

In [92]:
basin1Light.to_csv('bisin1_ligh.csv', index=None)

In [ ]:
### Making quadratic transformation in Labels for regression
DS = pd.read_csv('basin1TrainUnbalanced.csv', index_col = None)
print(Counter(DS['percentage']))

y_Quad = md.quadraticRechapeLabes(DS['percentage'], -0.125, 0.825)
DS['percentage'] = y_Quad
print(Counter(DS['percentage']))
DS.to_csv('basin1TrainUnbalanced_QuadTarget.csv', index = None)

## balanced sampling

In [ ]:
## Stratified Split
from sklearn.model_selection import StratifiedShuffleSplit

X,Y = ms.importDataSet('datasetBasin1_NoDataFree.csv', 'percentage')
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=50)
for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index.size, "TEST:", test_index.size)
    X_train = X.iloc[train_index]
    y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = Y.iloc[test_index]

In [ ]:
print(X_train.head())
X_train['percentage'] = y_train
print(X_train.head())
print(X_train.info())

In [ ]:
## This proportions are the reason why a sample_weight of 0.01 for the majority class give best results for regression
totalTrain = sum([1941324, 35056, 7250])
totalValidation = sum([485332,8764, 1812])
print("Balance summary of train dataset")
print(f"Class 1% : {1941324/totalTrain}, Class 1%: {7250/totalTrain}, Class 5%: {35056/totalTrain}")
print(f"Class 1% : {485332/totalValidation}, Class 1%: {1812/totalValidation}, Class 5%: {8764/totalValidation}")

In [ ]:
X,Y = ms.importDataSet('basin1TestForBanlanced.csv', 'percentage')
print('Resampled dataset shape %s' % Counter(Y))

In [ ]:
model = ms.loadModel('./outputs/2022-08-05/00-35-58/2208050035.pkl')
dataSetToSave = ms.makePredictionToImportAsSHP(csvName, model, X, Y, 'percentage')
print(dataSetToSave.head())

## Controled sampling

In [ ]:
DS = pd.read_csv('basin1TrainUnbalanced.csv', index_col = None)
print(DS.head())


In [ ]:
plt.boxplot(DS['FAcc']) # , , DS['elevation'], DS['disToRiv']]

In [ ]:
## Resampling appliying class selection by rule:

# RULE1: Select point at a distance to river less than 300m. 

# # newDS = pseudoClassCreation(DS, "distanceToRiver", 300, 2)
def pseudoClassCreation(dataset, conditionVariable, threshold, pseudoClass, targetClassName):
    '''
    Replace <targetClass> by  <pseudoClass> where <conditionVariable >= threshold>. 
    Return:
      dataset with new classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    conditionVar = (np.array(dataset[conditionVariable])).ravel()
    datsetReclassified[targetClassName] = [ pseudoClass if conditionVar[j] >= threshold 
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified

def revertPseudoClassCreation(dataset, originalClass, pseudoClass, targetClassName):
    '''
    Restablich  <targetClass> with <originalClass> where <targetClassName == pseudoClass>. 
    Return:
      dataset with original classes group. 
    '''
    datsetReclassified = dataset.copy()
    actualTarget = (np.array(dataset[targetClassName])).ravel()
    datsetReclassified[targetClassName] = [ originalClass if actualTarget[j] == pseudoClass
                                           else actualTarget[j]
                                           for j in range(len(actualTarget))]
    print(Counter(datsetReclassified[targetClassName]))
    return  datsetReclassified


print(Counter(X_train['percentage']))
newDS = pseudoClassCreation(X_train, 'disToRiv', 200, 2, 'percentage')
y = newDS['percentage']
newDS.drop(['percentage'], axis=1, inplace = True)
x_res,y_res = ms.randomUndersampling(newDS, y, )
x_res['percentage'] = y_res
# newDatase = revertPseudoClassCreation(x_res, 0, 2, 'percentage')


In [ ]:
x_res.to_csv('basin1ControlClass0Sampling4Class_ToSHP.csv',index = None)

# Data description and visualization

In [ ]:
### FAcc vs Labels
targets = DS['percentage']
FAcc = original['FAcc']
FAcc_norm = DS['FAcc_norm']
fig, axs = plt.subplots(1, 2, figsize=(13,4), sharey=True)
fig.text(-0.02, 0.5, 'labels', va='center', rotation='vertical')
fig.text(0.5, 1, 'Flow accumulation vs labels distribution', ha ='center')
axs[0].scatter(FAcc,targets)
# axs[0].set_title("Facc")
axs[0].set(xlabel='a) Flow Accumulation')
axs[1].scatter(FAcc_norm,targets)
# axs[1].set_title("FAcc_norm")
axs[1].set(xlabel='b) Flow Accumulation estandardized')
plt.rcParams['font.size'] = '20'
fig.tight_layout()


In [ ]:
##3 Al features vs labels
# 'disToRiv', 'TWI', 'TPI', 'slope', 'elevation',
targets = DS['percentage']
# targets = np.where( targets == 5, 2,targets)

E = DS['elevation'] 
slope = DS['slope']
FAcc = DS['FAcc_norm']
S = DS['disToRiv']
TWI = DS['TWI']
TPI = DS['TPI']

fig, axs = plt.subplots(3, 2, figsize=(13, 8), sharey=True)
fig.supylabel('Labels')
plt.rcParams['font.size'] = '15'
plt.yticks([0,1,5])
axs[0, 0].scatter(E,targets)
axs[0, 0].set_title("Elevation")
axs[1, 0].scatter(slope,targets)
axs[1, 0].set_title("Slope")
axs[2, 0].scatter(FAcc,targets)
axs[2, 0].set_title("Flow accumulation")
axs[0, 1].scatter(S,targets)
axs[0, 1].set_title("Distance to river")
axs[1, 1].scatter(TWI,targets)
axs[1, 1].set_title("TWI")
axs[2, 1].scatter(TPI,targets)
axs[2, 1].set_title('TPI')
fig.tight_layout()


In [ ]:
print(DS.head())
#  Return a dataset with the rows corresponding to the index where condition in DS.columName is valid. 
dsArray = DS[DS.percentage != 0] print(dsArray.head())
print(dsArray.head()) 

In [ ]:
sns.set(font_scale=1.5)
sns.pairplot(DS, hue = 'percentage', diag_kind = 'kde', 
             plot_kws = {'alpha': 0.8, 's': 100},
             height = 4, corner=True, palette = "Set2")# vars = ['life_exp', 'log_pop', 'log_gdp_per_cap'],

# sns.pairplot(DS, hue="percentage")

In [ ]:
####. Covariance Matrix
matrix = DS.corr().round(2)
sns.heatmap(matrix, annot=True)
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

estimator = RandomForestClassifier(criterion='entropy', random_state = 50)
x_train,y_train = ms.importDataSet('basin1Train.csv', 'percentage')
classifier = OneVsRestClassifier(estimator).fit(x_train,y_train)


In [ ]:
classifier = ms.loadModel('./outputs/2022-08-05/11-01-57/2208051101.pkl')
x_test,y_test = ms.importDataSet('basin1Test.csv', 'percentage')

x_test = ms.removeCoordinatesFromDataSet(x_test)

# y_prob = classifier.predict_proba(x_test)
#print(np.unique(y_prob))

In [ ]:
md.plot_ROC_AUC_OneVsRest(classifier, x_test, y_test)

In [ ]:
_,y_test = ms.importDataSet('./bestModels/Classifier/10-18-08/2208051018prediction_basin1Test.csv', 'prediction')
unique, count = np.unique(y_test, return_counts=True)
total = count.sum()
print(total)
percent = np.round(np.zeros_like(unique).astype('float16'),3)
print('values, counts , percent')
for i in range(len(unique)):    
   percent[i] = (count[i]/total)*100
   print(unique[i],"\t", count[i], percent[i])
